In [1]:
#### Kural Tabanlı Sınıflandırma ile Potansiyel Müşteri Getirisi Hesaplama ####

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
df = pd.read_csv("/kaggle/input/persona-1/persona_1.csv")

df.head()

,PRICE,SOURCE,SEX,COUNTRY,AGE
0,39,android,male,bra,17
1,39,android,male,bra,17
2,49,android,male,bra,17
3,29,android,male,tur,17
4,49,android,male,tur,17


In [3]:
##### GOREV - 1 #####
# Soru 1: persona.csv dosyasını okutunuz ve veri seti ile ilgili genel bilgileri gösteriniz.

df.shape
df.info()
df.columns
df.index
df.describe().T
df.isnull().values.any()

<class 'pandas.core.frame.DataFrame'>
Index: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   PRICE    5000 non-null   int64 
 1   SOURCE   5000 non-null   object
 2   SEX      5000 non-null   object
 3   COUNTRY  5000 non-null   object
 4   AGE      5000 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 234.4+ KB


False

In [4]:
# Soru 2: Kaç unique SOURCE vardır? Frekansları nedir

df["SOURCE"].unique()
df["SOURCE"].nunique()
df["SOURCE"].value_counts()

SOURCE
android    2974
ios        2026
Name: count, dtype: int64

In [5]:
# Soru 3: Kaç unique PRICE vardır?
df["PRICE"].nunique()

6

In [6]:
# Soru 4: Hangi PRICE'dan kaçar tane satış gerçekleşmiş
df["PRICE"].value_counts()

PRICE
29    1305
39    1260
49    1031
19     992
59     212
9      200
Name: count, dtype: int64

In [7]:
# Soru 5: Hangi ülkeden kaçar tane satış olmuş?
df["COUNTRY"].value_counts()

COUNTRY
usa    2065
bra    1496
deu     455
tur     451
fra     303
can     230
Name: count, dtype: int64

In [8]:
# Soru 6: Ülkelere göre satışlardan toplam ne kadar kazanılmış
df.groupby("COUNTRY").agg({"PRICE" : "sum"})

,PRICE
COUNTRY,
bra,51354
can,7730
deu,15485
fra,10177
tur,15689
usa,70225


In [9]:
# Soru 7: SOURCE türlerine göre satış sayıları nedir --???
df["SOURCE"].value_counts()


SOURCE
android    2974
ios        2026
Name: count, dtype: int64

In [10]:
# Soru 8: Ülkelere göre PRICE ortalamaları nedir?
df.groupby("COUNTRY").agg({"PRICE" : "mean"})

,PRICE
COUNTRY,
bra,34.327540
can,33.608696
deu,34.032967
fra,33.587459
tur,34.787140
usa,34.007264


In [11]:
# Soru 9: SOURCE'lara göre PRICE ortalamaları nedir?
df.groupby("SOURCE").agg({"PRICE" : "mean"})

,PRICE
SOURCE,
android,34.174849
ios,34.069102


In [12]:
# Soru 10: COUNTRY-SOURCE kırılımında PRICE ortalamaları nedir?
df.groupby(["COUNTRY", "SOURCE"]).agg({"PRICE" : "mean"})

PRICE
COUNTRY SOURCE            
bra     android  34.387029
        ios      34.222222
can     android  33.330709
        ios      33.951456
deu     android  33.869888
        ios      34.268817
fra     android  34.312500
        ios      32.776224
tur     android  36.229437
        ios      33.272727
usa     android  33.760357
        ios      34.371703

In [13]:
##### GOREV - 2 #####

#  COUNTRY, SOURCE, SEX, AGE kırılımında ortalama kazançlar nedir?
agg_df = df.groupby(["COUNTRY","SOURCE", "SEX", "AGE"]).agg({"PRICE" : "mean"}).sort_values(by="PRICE", ascending=False)

agg_df = agg_df.reset_index()

agg_df["NEW_AGE"] = pd.cut(agg_df["AGE"], [0, 18, 23, 30, 40, agg_df["AGE"].max()],
                           labels=['0_18', '19_23', '24_30', '31_40', '41_' + str(agg_df["AGE"].max())])
agg_df.head()



,COUNTRY,SOURCE,SEX,AGE,PRICE,NEW_AGE
0,bra,android,male,46,59.0,41_66
1,usa,android,male,36,59.0,31_40
2,fra,android,female,24,59.0,24_30
3,usa,ios,male,32,54.0,31_40
4,deu,android,female,36,49.0,31_40


In [14]:
agg_df['customers_level_based'] = agg_df[['COUNTRY', 'SOURCE', 'SEX', 'NEW_AGE']].agg(lambda x: '_'.join(x).upper(), axis=1)
agg_df.head()


,COUNTRY,SOURCE,SEX,AGE,PRICE,NEW_AGE,customers_level_based
0,bra,android,male,46,59.0,41_66,BRA_ANDROID_MALE_41_66
1,usa,android,male,36,59.0,31_40,USA_ANDROID_MALE_31_40
2,fra,android,female,24,59.0,24_30,FRA_ANDROID_FEMALE_24_30
3,usa,ios,male,32,54.0,31_40,USA_IOS_MALE_31_40
4,deu,android,female,36,49.0,31_40,DEU_ANDROID_FEMALE_31_40


In [15]:
agg_df = agg_df.groupby("customers_level_based").agg({"PRICE" : "mean"}).sort_values(by="PRICE", ascending=False).reset_index()
agg_df.head()

,customers_level_based,PRICE
0,FRA_ANDROID_FEMALE_24_30,45.428571
1,TUR_IOS_MALE_24_30,45.000000
2,TUR_IOS_MALE_31_40,42.333333
3,TUR_ANDROID_FEMALE_31_40,41.833333
4,CAN_ANDROID_MALE_19_23,40.111111


In [16]:
agg_df["SEGMENT"] = pd.qcut(agg_df["PRICE"], 4, labels=["D", "C", "B", "A"])
agg_df.head()

,customers_level_based,PRICE,SEGMENT
0,FRA_ANDROID_FEMALE_24_30,45.428571,A
1,TUR_IOS_MALE_24_30,45.000000,A
2,TUR_IOS_MALE_31_40,42.333333,A
3,TUR_ANDROID_FEMALE_31_40,41.833333,A
4,CAN_ANDROID_MALE_19_23,40.111111,A


In [17]:
segmentSummary = agg_df.groupby("SEGMENT").agg({"PRICE" : ["mean", "max", "sum"]}).reset_index()
segmentSummary.head()

/tmp/ipykernel_17/3173880524.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  segmentSummary = agg_df.groupby("SEGMENT").agg({"PRICE" : ["mean", "max", "sum"]}).reset_index()


SEGMENT      PRICE                        
                mean        max          sum
0       D  29.206780  32.333333   817.789833
1       C  33.509674  34.077340   904.761209
2       B  34.999645  36.000000   944.990411
3       A  38.691234  45.428571  1044.663328

In [18]:
# TUR_ANDROID_FEMALE_31_40 
# 33 yaşında ANDROID kullanan bir Türk kadını hangi segmentea ittir ve ortalama ne kadar gelir kazandırması beklenir?

agg_df[agg_df["customers_level_based"] == "TUR_ANDROID_FEMALE_31_40"]


,customers_level_based,PRICE,SEGMENT
3,TUR_ANDROID_FEMALE_31_40,41.833333,A
